# Set-Up
System dependencies

In [ ]:
# be able to take input from a server (sent from mobile device) or from the camera sensor.

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# importing dependencies
import cv2
from matplotlib import pyplot as plt
import imghdr

In [ ]:
# Set the data directory
data_dir = 'data'

In [ ]:
# telling the computer what the image extensions are
image_exts = ['jpeg', 'jpg', 'bmp', 'png'] 

In [ ]:
# os.listdir(data_dir) # should return benign and malignant ; 
# returns folders inside of the data dir

In [ ]:
# os.listdir(os.path.join(data_dir, 'malignant'))
# os.listdir(os.path.join(data_dir, 'benign'))
# returns every image within the folder

In [ ]:
for image_class in os.listdir(data_dir):
    print(image_class) # loops thru directories

In [ ]:
# cv2.imread(os.path.join('data', 'malignant', '267.png'))
# type(img) numpy array
# img.shape # 3 means is is a colored image long x high
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
for image_class in os.listdir(data_dir):
    if image_class != '.DS_Store': # i'm not sure what .DS_Store is doing here
        for image in os.listdir(os.path.join(data_dir, image_class)):
            image_path = os.path.join(data_dir, image_class, image)
            try:
                img = cv2.imread(image_path) # opens image
                tip = imghdr.what(image_path)
                if tip not in image_exts:
                    print("Image not in ext list {}".format(image_path))
                    os.remove(image_path) # deletes files from the dir if they r weird
            except Exception as e:
                print('Issue with image {}'.format(image_path)) # should probably change this to printf